<a href="https://colab.research.google.com/github/chrishayuk/opl-train/blob/main/Fine_Tune_OPL_in_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # install dependencies

# # we use the latest version of transformers, peft, and accelerate
# !pip install -q accelerate peft transformers

# # install bitsandbytes for quantization
# !pip install -q bitsandbytes

# # install trl for the SFT library
# !pip install -q trl

# # we need sentencepiece for the llama2 slow tokenizer
# !pip install sentencepiece

# # we need einops, used by falcon-7b, llama-2 etc
# # einops (einsteinops) is used to simplify tensorops by making them readable
# !pip install -q -U einops

# # we need to install datasets for our training dataset
# !pip install -q datasets


# ==================================================
# !pip install protobuf

# !pip install -q ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension

# pip install -q tensorboardX

## Settings
The following configures our settings for finetuning our model

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "isma77777/data-version2" ## should be replaced by our real dataset in our final fine-tuning

# Fine-tuned model name
new_model = "llama-2-7b-test"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 5

## Download the base model
The following will download the base model, in this case the llama-2-7b-chat-hf model.

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

model.config.pretraining_tp = 1
# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/tbab

# Train the Model
The following section is about taking your dataset and then finetuning the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model

In [4]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

## Fine Tune the Model
The following section will take your dataset, and fine tune the model with it.

In [5]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=1,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=25,                           # we're not gonna save
    logging_steps=2,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/9695 [00:00<?, ?it/s]

{'loss': 1.173, 'grad_norm': 0.11466719955205917, 'learning_rate': 1.3745704467353952e-06, 'epoch': 0.0}
{'loss': 1.2677, 'grad_norm': 0.08691975474357605, 'learning_rate': 2.7491408934707903e-06, 'epoch': 0.0}
{'loss': 1.4166, 'grad_norm': 0.10017478466033936, 'learning_rate': 4.123711340206186e-06, 'epoch': 0.0}
{'loss': 1.2465, 'grad_norm': 0.1125314012169838, 'learning_rate': 5.498281786941581e-06, 'epoch': 0.0}
{'loss': 1.1107, 'grad_norm': 0.08278153091669083, 'learning_rate': 6.872852233676977e-06, 'epoch': 0.01}
{'loss': 1.2682, 'grad_norm': 0.07678315788507462, 'learning_rate': 8.247422680412371e-06, 'epoch': 0.01}
{'loss': 1.5926, 'grad_norm': 0.1144038662314415, 'learning_rate': 9.621993127147768e-06, 'epoch': 0.01}
{'loss': 1.3504, 'grad_norm': 0.11382138729095459, 'learning_rate': 1.0996563573883161e-05, 'epoch': 0.01}
{'loss': 1.2404, 'grad_norm': 0.1098538264632225, 'learning_rate': 1.2371134020618558e-05, 'epoch': 0.01}
{'loss': 1.5416, 'grad_norm': 0.12706245481967926,

Checkpoint destination directory ./results/checkpoint-25 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 1.6752, 'grad_norm': 0.1406734734773636, 'learning_rate': 1.7869415807560138e-05, 'epoch': 0.01}
{'loss': 1.6335, 'grad_norm': 0.15578381717205048, 'learning_rate': 1.9243986254295536e-05, 'epoch': 0.01}
{'loss': 1.8391, 'grad_norm': 0.19295527040958405, 'learning_rate': 2.0618556701030927e-05, 'epoch': 0.02}
{'loss': 1.9789, 'grad_norm': 0.2170020490884781, 'learning_rate': 2.1993127147766322e-05, 'epoch': 0.02}
{'loss': 1.9645, 'grad_norm': 0.24052855372428894, 'learning_rate': 2.336769759450172e-05, 'epoch': 0.02}
{'loss': 1.9646, 'grad_norm': 0.24929283559322357, 'learning_rate': 2.4742268041237116e-05, 'epoch': 0.02}
{'loss': 2.23, 'grad_norm': 0.30085790157318115, 'learning_rate': 2.611683848797251e-05, 'epoch': 0.02}
{'loss': 2.2003, 'grad_norm': 0.34330621361732483, 'learning_rate': 2.749140893470791e-05, 'epoch': 0.02}
{'loss': 2.7785, 'grad_norm': 0.3460121750831604, 'learning_rate': 2.8865979381443297e-05, 'epoch': 0.02}
{'loss': 2.7345, 'grad_norm': 0.4377850890159

...

{'loss': 0.5662, 'grad_norm': 0.268219530582428, 'learning_rate': 0.00019775837630582266, 'epoch': 0.48}
{'loss': 0.6439, 'grad_norm': 0.3614184558391571, 'learning_rate': 0.00019774428701511734, 'epoch': 0.48}
{'loss': 0.6035, 'grad_norm': 0.3100531995296478, 'learning_rate': 0.00019773015409032805, 'epoch': 0.48}
{'loss': 0.7946, 'grad_norm': 0.27295583486557007, 'learning_rate': 0.00019771597753776395, 'epoch': 0.48}
{'loss': 0.5473, 'grad_norm': 0.4462517201900482, 'learning_rate': 0.00019770175736375358, 'epoch': 0.48}
{'loss': 0.4665, 'grad_norm': 0.37601613998413086, 'learning_rate': 0.00019768749357464493, 'epoch': 0.48}
{'loss': 0.7105, 'grad_norm': 0.3236527442932129, 'learning_rate': 0.00019767318617680554, 'epoch': 0.48}
{'loss': 0.8024, 'grad_norm': 0.4720814824104309, 'learning_rate': 0.0001976588351766224, 'epoch': 0.48}
{'loss': 0.8327, 'grad_norm': 0.4985937774181366, 'learning_rate': 0.00019764444058050193, 'epoch': 0.49}
{'loss': 0.8443, 'grad_norm': 0.46524110436439

Checkpoint destination directory ./results/checkpoint-950 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 1.0356, 'grad_norm': 1.3305107355117798, 'learning_rate': 0.0001975864263654566, 'epoch': 0.49}
{'loss': 0.7353, 'grad_norm': 0.19739165902137756, 'learning_rate': 0.0001975718138864271, 'epoch': 0.49}
{'loss': 0.8394, 'grad_norm': 0.3823391795158386, 'learning_rate': 0.00019755715785030754, 'epoch': 0.49}
{'loss': 0.5209, 'grad_norm': 0.2174692004919052, 'learning_rate': 0.00019754245826364043, 'epoch': 0.49}
{'loss': 0.6693, 'grad_norm': 0.23924972116947174, 'learning_rate': 0.0001975277151329879, 'epoch': 0.49}
{'loss': 0.6855, 'grad_norm': 0.2242138534784317, 'learning_rate': 0.00019751292846493138, 'epoch': 0.49}
{'loss': 0.7057, 'grad_norm': 0.2608100175857544, 'learning_rate': 0.00019749809826607187, 'epoch': 0.5}
{'loss': 0.7382, 'grad_norm': 0.317317932844162, 'learning_rate': 0.0001974832245430297, 'epoch': 0.5}
{'loss': 0.6876, 'grad_norm': 0.245153546333313, 'learning_rate': 0.00019746830730244463, 'epoch': 0.5}
{'loss': 0.4626, 'grad_norm': 0.17401063442230225, 'l

/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 377934a8-c090-4c9b-aacd-bb8ff47daff5)') - silently ignoring the lookup for the file config.json in NousResearch/Llama-2-7b-chat-hf.
  warnings.warn(
/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in NousResearch/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.4178, 'grad_norm': 0.5726067423820496, 'learning_rate': 7.837349531182536e-06, 'epoch': 4.38}
{'loss': 0.3986, 'grad_norm': 0.5874629616737366, 'learning_rate': 7.81144106977766e-06, 'epoch': 4.38}
{'loss': 0.4157, 'grad_norm': 0.7348241806030273, 'learning_rate': 7.78557376232093e-06, 'epoch': 4.39}
{'loss': 0.4586, 'grad_norm': 1.0093730688095093, 'learning_rate': 7.759747620359758e-06, 'epoch': 4.39}
{'loss': 0.3393, 'grad_norm': 0.2546720504760742, 'learning_rate': 7.73396265542322e-06, 'epoch': 4.39}
{'loss': 0.2296, 'grad_norm': 0.19211259484291077, 'learning_rate': 7.708218879022022e-06, 'epoch': 4.39}
{'loss': 0.2485, 'grad_norm': 0.22691130638122559, 'learning_rate': 7.682516302648423e-06, 'epoch': 4.39}
{'loss': 0.3844, 'grad_norm': 0.31191185116767883, 'learning_rate': 7.656854937776326e-06, 'epoch': 4.39}
{'loss': 0.3834, 'grad_norm': 0.3476581275463104, 'learning_rate': 7.631234795861265e-06, 'epoch': 4.39}
{'loss': 0.2902, 'grad_norm': 0.2884020209312439, 'lear

# Run the Model
The following runs the model post fine tune

In [6]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model

prompt = "Write a code code scenario of a data encryption program using SHA-256 algorithm"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] Write a code code scenario of a data encryption program using SHA-256 algorithm [/INST] #define DEBUG 0\nint main(int argc, char **argv)\n{\n#if DEBUG\n    printf("debug enabled\\n");\n#endif\n    unsigned int i;\n    unsigned int j;\n    unsigned int k;\n    unsigned int num_blocks;\n    unsigned int num_bytes;\n    unsigned int num_key_bits;\n    unsigned int num_block_key_bits;\n    unsigned int num_key_bits;\n    unsigned int num_block_key_bits;\n    unsigned int num_key_bits_mod;\n    unsigned int num_block_key_bits_mod;\n    unsigned int num_key_bits_mod;\n    unsigned int num_block_key_bits_mod;\n    unsigned int


In [7]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [8]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [9]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0}#device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/tbabanaerep/miniconda3/envs/env_torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
# !huggingface-cli login

# model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)